In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
print(df.shape)
print(df['Date'].head(100))

NameError: name 'df' is not defined

In [ ]:
# Load the data
df = pd.read_csv('lse_stocks_comprehensive_data.csv')

# Display basic information about the dataset
#print(df.info)

# Check for missing values
print("\nMissing values:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# Check for non-numeric columns
non_numeric = df.select_dtypes(exclude=[np.number]).columns
print("\nNon-numeric columns:", non_numeric.tolist())

# Encode categorical variables
if False: 
    le = LabelEncoder()
    for col in non_numeric:
        df[col] = le.fit_transform(df[col].astype(str))

# Check for infinite values
print("\nInfinite values:")
print(np.isinf(df.select_dtypes(include=[np.number])).sum())

# Display summary statistics
print("\nSummary statistics:")
print(df.describe())

# Check for low variance features
low_variance = df.var()[df.var() < 0.1].index.tolist()
print("\nLow variance features:", low_variance)

# Check for highly correlated features
correlation_matrix = df.corr()
high_correlation = np.where(np.abs(correlation_matrix) > 0.95)
high_correlation = [(correlation_matrix.index[x], correlation_matrix.columns[y]) for x, y in zip(*high_correlation) if x != y and x < y]
#print("\nHighly correlated features:")
#print(high_correlation)

print("\nThe data is ready for XGBoost if there are no missing values, infinite values, or other issues highlighted above.")



Missing values:
income_stmt:Normalized EBITDA                        503
income_stmt:Reconciled Cost Of Revenue               503
income_stmt:EBITDA                                   503
income_stmt:EBIT                                     503
income_stmt:Total Expenses                           503
                                                    ... 
cashflow:Earnings Losses From Equity Investments    2511
balance_sheet:Preferred Stock Equity                2513
balance_sheet:Preferred Stock                       2513
cashflow:Preferred Stock Dividend Paid              2513
cashflow:Short Term Debt Issuance                   3014
Length: 182, dtype: int64

Non-numeric columns: ['Date', 'Ticker']

Infinite values:
index                                               0
Open                                                0
High                                                0
Low                                                 0
Close                                               0
 

TypeError: could not convert string to float: '2022-09-20'

In [ ]:
import pandas as pd
import numpy as np

# Assuming you have already loaded your dataframe 'df'

# Check if all required columns exist
required_columns = ['income_stmt:EBIT', 'Date', 'Ticker']
if all(col in df.columns for col in required_columns):
    # Get the rows where 'income_stmt:EBIT' is null
    missing_ebit_rows = df[df['income_stmt:EBIT'].isnull()][['Date', 'Ticker']]
    
    # If there are any missing values
    if not missing_ebit_rows.empty:
        # Get 10 random rows (or less if there are fewer than 10 missing values)
        random_rows = missing_ebit_rows.sample(n=min(10, len(missing_ebit_rows)))
        
        print("10 random entries with missing values in 'income_stmt:EBIT' column:")
        for index, row in random_rows.iterrows():
            print(f"Date: {row['Date']}, Ticker: {row['Ticker']}")
        
        print(f"\nTotal number of missing 'income_stmt:EBIT' values: {len(missing_ebit_rows)}")
    else:
        print("There are no missing values in the 'income_stmt:EBIT' column.")
else:
    missing_cols = [col for col in required_columns if col not in df.columns]
    print(f"The following required columns are missing from the dataframe: {', '.join(missing_cols)}")

# Optionally, print the data types of the columns
print("\nData types of the columns:")
print(df[required_columns].dtypes)


10 random entries with missing values in 'income_stmt:EBIT' column:
Date: 2024-02-12, Ticker: HSBA
Date: 2023-06-12, Ticker: HSBA
Date: 2023-10-27, Ticker: HSBA
Date: 2023-04-28, Ticker: HSBA
Date: 2024-01-11, Ticker: HSBA
Date: 2023-02-03, Ticker: HSBA
Date: 2024-05-10, Ticker: HSBA
Date: 2024-07-17, Ticker: HSBA
Date: 2023-01-06, Ticker: HSBA
Date: 2023-04-24, Ticker: HSBA

Total number of missing 'income_stmt:EBIT' values: 503

Data types of the columns:
income_stmt:EBIT    float64
Date                 object
Ticker               object
dtype: object


In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('lse_stocks_comprehensive_data.csv')

# Group by 'Ticker' and sample 8 rows from each group
sampled_df = df.groupby('Ticker').apply(lambda x: x.sample(n=min(8, len(x)), random_state=42)).reset_index(drop=True)

# Save the sampled DataFrame to a new CSV file
sampled_df.to_csv('lse_stocks_sampled_data.csv', index=False)

print(f"Original CSV shape: {df.shape}")
print(f"Sampled CSV shape: {sampled_df.shape}")
print(f"Number of unique Tickers in sampled data: {sampled_df['Ticker'].nunique()}")


Original CSV shape: (3014, 288)
Sampled CSV shape: (48, 288)
Number of unique Tickers in sampled data: 6


C:\Users\User\AppData\Local\Temp\ipykernel_10456\997944792.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('Ticker').apply(lambda x: x.sample(n=min(8, len(x)), random_state=42)).reset_index(drop=True)


In [ ]:
import pandas as pd

# Assuming you have already loaded your dataframe 'df'

# Get the count of missing values for each column
missing_values = df.isnull().sum()

# Filter for columns with missing values
columns_with_missing = missing_values[missing_values > 0]

# Print the results
print("Columns with missing values:")
print(columns_with_missing)

# Get the counts for 'date' and 'ticker' columns for rows with missing values
if 'Date' in df.columns and 'Ticker' in df.columns:
    missing_by_date = df[df.isnull().any(axis=1)]['Date'].value_counts()
    missing_by_ticker = df[df.isnull().any(axis=1)]['Ticker'].value_counts()

    #print("\nMissing values count by date:")
    #print(missing_by_date)
    
    print("\nMissing values count by ticker:")
    print(missing_by_ticker)
else:
    print("\nEither 'date' or 'ticker' column is not present in the dataframe.")


Columns with missing values:
income_stmt:Normalized EBITDA                        503
income_stmt:Reconciled Cost Of Revenue               503
income_stmt:EBITDA                                   503
income_stmt:EBIT                                     503
income_stmt:Total Expenses                           503
                                                    ... 
cashflow:Earnings Losses From Equity Investments    2511
balance_sheet:Preferred Stock Equity                2513
balance_sheet:Preferred Stock                       2513
cashflow:Preferred Stock Dividend Paid              2513
cashflow:Short Term Debt Issuance                   3014
Length: 182, dtype: int64

Missing values count by ticker:
Ticker
AZN     503
HSBA    503
BHP     503
SHEL    502
ULVR    502
BP      501
Name: count, dtype: int64


In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
print(df.columns.values)

['index' 'Date' 'Open' 'High' 'Low' 'Close' 'Volume' 'Dividends'
 'Stock Splits' 'income_stmt:Tax Effect Of Unusual Items'
 'income_stmt:Tax Rate For Calcs' 'income_stmt:Normalized EBITDA'
 'income_stmt:Total Unusual Items'
 'income_stmt:Total Unusual Items Excluding Goodwill'
 'income_stmt:Net Income From Continuing Operation Net Minority Interest'
 'income_stmt:Reconciled Depreciation'
 'income_stmt:Reconciled Cost Of Revenue' 'income_stmt:EBITDA'
 'income_stmt:EBIT' 'income_stmt:Net Interest Income'
 'income_stmt:Interest Expense' 'income_stmt:Interest Income'
 'income_stmt:Normalized Income'
 'income_stmt:Net Income From Continuing And Discontinued Operation'
 'income_stmt:Total Expenses' 'income_stmt:Rent Expense Supplemental'
 'income_stmt:Total Operating Income As Reported'
 'income_stmt:Diluted Average Shares' 'income_stmt:Basic Average Shares'
 'income_stmt:Diluted EPS' 'income_stmt:Basic EPS'
 'income_stmt:Diluted NI Availto Com Stockholders'
 'income_stmt:Net Income Common S